In [46]:
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import LeaveOneOut
from os import listdir

In [ ]:
#SET PATH
#Input takes 'locally' or 'remotely' and returns an absolute path based on the input 
location=input('Are you working locally or remotely today?')

if location=='locally':
    path="/Users/cameronkelsey/Documents/smack_lab/cayo_data/"
elif location=='remotely':
    path="/scratch/ckelsey4/Cayo_meth/"

In [52]:
#Import metadata
long_data=pd.read_csv(path + "long_data_adjusted.txt", sep="\t")
long_data=long_data[long_data['n'] > 1]
long_data=long_data.sort_values(by=['lid_pid'])

In [5]:
#Define natural sorting function
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

In [ ]:
#Import coverage data per chromosome
dnam_path=path + "dnam_clock/"

cov_files=[f for f in listdir(dnam_path)
        if "cov" in f]

cov_files=sorted(cov_files, key=natural_key)        

cov_full=[]

for file in cov_files:
        file=dnam_path + file        
        dd=pd.read_csv(file, sep=" ")
        cov_full.append(dd)

#Remove last instance of dd to free up memory
del dd

In [10]:
#Import count data per chromosome
m_files=[f for f in listdir(dnam_path)
        if "m" in f]

m_files=sorted(m_files, key=natural_key)        

m_full=[]

for file in m_files:
        file=dnam_path + file        
        dd=pd.read_csv(file, sep=" ")
        m_full.append(dd)

#Remove last instance of dd to free up memory
del dd

In [54]:
#Import M and Cov for regions
#Import Cov and subset cols to lid_pid in metadata
regions_cov=pd.read_csv(dnam_path + "regions_cov.txt", sep=" ")
regions_cov=regions_cov[regions_cov.columns.intersection(long_data['lid_pid'])]
                        
regions_m=pd.read_csv(dnam_path + "regions_m.txt", sep=" ")
regions_m=regions_m[regions_m.columns.intersection(regions_cov.columns)]

#Match metadata columns to M and Cov matrices
long_data=long_data[long_data['lid_pid'].isin(regions_cov.columns)]

In [56]:
#Check that col names for M, Cov and metadata lid_pid are all equal and generate %methylation matrix
if all(regions_m.columns == regions_cov.columns) == True and all(regions_m.columns == long_data['lid_pid']) == True:
    p_meth=regions_m/regions_cov
    p_meth=p_meth.fillna(0)
    print("M and Cov columns match. Col LIDs and metadata LIDs match. %methylation matrix generated. NAs replaced with 0's. Onwards!")


M and Cov columns match. Col LIDs and metadata LIDs match. %methylation matrix generated. NAs replaced with 0's. Onwards!


In [ ]:
#Remove X-chromosome from p_meth
#Generate list of chrs where n=n of regions
chrs=[i[0] for i in p_meth.index.str.split('_', n=1)]
p_meth['chr']=chrs

#Filter out X-chrom
p_meth=p_meth[p_meth['chr'] != 'X']

#Print list of reminaing chrs to be sure
print(test['chr'].unique())

#Remove chr col from p_meth
p_meth=p_meth.drop('chr', axis=1)


In [47]:
cv=LeaveOneOut()
cv.get_n_splits(regions_m)

118001